In [1]:
import numpy as np
from scipy.optimize import differential_evolution, LinearConstraint, Bounds
from scipy.special import softmax
import modeling

def my_objective_func(params, D):
	# unpack the parameters
	[alpha_2, beta, beta_meta, concentration_2, epsilon, prior_1, prior_2] = params
	if alpha_2 < 0 or epsilon < 0 or prior_1 < 0 or prior_2 < 0:
		print(f"alpha_2: {alpha_2}, beta: {beta}, beta_meta: {beta_meta}, concentration_2: {concentration_2}, epsilon: {epsilon}, prior_1: {prior_1}, prior_2: {prior_2}")
	beta_2 = beta
	beta_policies = 5
	beta_meta = 10**beta_meta
	concentration_2 = 10**concentration_2
	
	# initialize variables
	llh = 0
	num_block = 12
	s_2 = a_2 = -1
	block = -1

	nTS_2 = 2 # number of task-sets in stage2
	TS_2s = np.ones((nTS_2,2,4)) / 4 # Q-values for each TS in stage2
	nC_2 = 2 * num_block # number of contexts in stage2
	PTS_2 = np.zeros((nTS_2,nC_2)) # probability of choosing each TS in each context
	PTS_2[0,0::2] = 1
	PTS_2[1,1::2] = 1
	encounter_matrix_2 = np.zeros(nC_2) # whether a context has been encountered
	encounter_matrix_2[:nTS_2] = 1
	p_policies = np.array([prior_1, prior_2, 1-prior_1-prior_2]) # probability of sampling each policy
	p_policies_softmax = softmax(beta_policies * p_policies) # softmax transform of the policy probabilities

	for t in range(D.shape[0]):	# loop over all trials
		stage = int(D[t,1])

		if int(D[t,5]) == 1: # new block
			block += 1

		if stage == 1: # skip stage1
			s_1 = int(D[t, 2])
			actions_tried = set()
		elif stage == 2: # stage2
			# get stim, action, reward info
			s_2 = int(D[t, 2])
			a_2 = int(D[t, 3]) - 4
			r_2 = int(D[t, 4])

			# get the context and state, determined by model structure
			cue = s_2
			state = s_1
			c_2 = block * 2 + cue # The context of stage2
			c_2_alt = block * 2 + (1 - cue) # The alternative context of stage2
			# update the PTS matrix with newly encountered context, if any
			for this_c_2 in sorted([c_2, c_2_alt]):
				if encounter_matrix_2[this_c_2] == 0:
					if this_c_2 > 0:
						specs = PTS_2.shape
						PTS_2[:,this_c_2] = np.sum(this_c_2[:,:(this_c_2//2)*2], axis=1)
						if np.sum(PTS_2[:,c]) > 0:
							PTS_2[:,this_c_2] /= np.sum(this_c_2[:,this_c_2])
							new_PTS = np.zeros((specs[0]+1,specs[1]))
							new_PTS[:-1,:] = PTS_2 
							new_PTS[-1,this_c_2] = concentration_2 # create new task set
							new_PTS[:,this_c_2] /= np.sum(new_PTS[:,this_c_2]) # normalize the probability distribution over the set of TS
						PTS_2 = new_PTS
						TS_2s = np.vstack((TS_2s, [np.ones((2,4)) / 4])) # initialize Q-values for new TS creation
						nTS_2 += 1
					encounter_matrix_2[this_c_2] = 1

			# update the biases matrix for context-pairing
			biases = np.zeros((nTS_2, nTS_2))
			for i in range(block):
				this_TS_1 = np.argmax(PTS_2[:-2,i*2])
				this_TS_2 = np.argmax(PTS_2[:-2,i*2+1])
				biases[this_TS_1, this_TS_2] += 1
				biases[this_TS_2, this_TS_1] += 1

			# incorporate the biases into task-set probabilities
			TS_2_alt = np.argmax(PTS_2[:,c_2_alt])
			if block > 0:
				bias = biases[TS_2_alt].copy()
				b = np.max(PTS_2[:,c_2_alt])
				if np.sum(bias) > 0 and np.max(PTS_2[:,c_2]) < 0.5:
					bias /= np.sum(bias)
					PTS_2[:,c_2] = PTS_2[:,c_2] * (1 - b) + bias * b

			# sample a task-set based on the TS probabilities given the context
			Q_full = TS_2s[:, state].copy()
			if len(actions_tried) > 0:
				Q_full[:,list(actions_tried)] = -1e20
			pchoice_2_full = softmax(beta_2 * Q_full, axis=-1)
			pchoice_2_full = np.sum(pchoice_2_full[:,a_2-1] * PTS_2[:,c_2]) * (1-epsilon) + epsilon / 4

			# compute the choice policy
			Q_compress_1 = np.mean(TS_2s, axis=(1)) # compressed policy over stage1
			Q_compress_2 = (TS_2s + np.sum(TS_2s * PTS_2[:,c_2_alt].reshape(-1,1,1),axis=0))[:,state] / 2 # compressed policy over stage2
            
            # avoid choosing the same action in the same stage of the same trial
			if len(actions_tried) > 0:
				Q_compress_1[:,list(actions_tried)] = -1e20
				Q_compress_2[:,list(actions_tried)] = -1e20
            # compute the choice policies for compressed policies
			pchoice_2_compress_1 = softmax(beta_2 * Q_compress_1, axis=-1)
			pchoice_2_compress_1 = np.sum(pchoice_2_compress_1[:,a_2-1] * PTS_2[:,c_2]) * (1-epsilon) + epsilon / 4
			pchoice_2_compress_2 = softmax(beta_2 * Q_compress_2, axis=-1) 
			pchoice_2_compress_2 = np.sum(pchoice_2_compress_2[:,a_2-1] * PTS_2[:,c_2]) * (1-epsilon) + epsilon / 4
            # take a weighted sum of the two compressed policies and the fully hierarchical policy
			pchoice_2 = p_policies_softmax[0] * pchoice_2_compress_1 \
			            + p_policies_softmax[1] * pchoice_2_compress_2 \
			            + p_policies_softmax[2] * pchoice_2_full

			# compute the negative log likelihood of the choice based on the choice policy
			if np.isnan(pchoice_2) or pchoice_2 <= 0:
				print(f"pchoice_2: {pchoice_2}")
				print(f"prior_1: {prior_1}, prior_2: {prior_2}, epsilon: {epsilon}")
				print(f"p_policies_softmax: {p_policies_softmax}")
				print(f"pchoice_2_compress_1: {pchoice_2_compress_1}")
				print(f"pchoice_2_compress_2: {pchoice_2_compress_2}")
				print(f"pchoice_2_full: {pchoice_2_full}")
			llh += np.log(pchoice_2)
			correct_2 = r_2

			# update the task-set probabilities based on the choice and the reward using Bayes Rule
			PTS_2[:,c_2] *= (1 - correct_2 - (-1)**correct_2 * TS_2s[:, state, a_2-1])
			PTS_2[:,c_2] += 1e-6
			PTS_2[:,c_2] /= np.sum(PTS_2[:,c_2])

			# update the Q-values of the task-sets based on the reward prediction error
			RPE = (r_2 - TS_2s[:,state,a_2-1]) * PTS_2[:,c_2]
			TS_2s[:,state,a_2-1] += alpha_2 * RPE

			# update the policy probabilities using Bayes Rule
			likelihoods = np.array([pchoice_2_compress_1, pchoice_2_compress_2, pchoice_2_full])
			likelihoods = softmax(beta_meta * likelihoods)
			p_policies *= (1 - correct_2 - (-1)**correct_2 * likelihoods)
			if np.min(p_policies) < 1e-6:
				p_policies += 1e-6
			p_policies /= np.sum(p_policies)
			p_policies_softmax = softmax(beta_policies * p_policies)

			actions_tried.add(a_2-1)

	return -llh

# np.random.seed(2848769375)
X = np.load('D.npy')
constraints = LinearConstraint([0,0,0,0,0,1,1], lb=2e-6, ub=1-1e-6, keep_feasible=True)
bounds = Bounds(lb=[1e-6, 1e-6, -6, -2, 1e-6, 1e-6, 1e-6], ub=[1, 20, 0.5, 0.7, 1, 1, 1])
result = differential_evolution(func=modeling.abstraction_model_nllh, bounds=bounds, constraints=constraints, args=(X, 'backward', True))

/home/exouser/anaconda3/envs/OT/lib/python3.8/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


In [4]:
current_seed = np.random.get_state()[1][0]
print(f"Current random seed: {current_seed}")

Current random seed: 2848769375
